# 毕业作业第一稿

### 本稿主要是增加衍生变量，采用baseline算法，看结果是否有提高

### 根据第七章作业构建衍生变量思路：
### 在观察数据特征时发现：对于以下离散变量：
### discrete_grade_2_one_hot
### discrete_grade_4_one_hot
### discrete_grade_5_one_hot
### discrete_grade_6_one_hot
###  在loan_status取不同值时，差别较大，故利用这些变量构造衍生变量

In [12]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [5]:
x_temp1 = pd.read_csv('E:\\极客大学\\机器学习训练营\\作业\毕业设计\\ML_chapter7_dataset\\final\\train_final.csv', engine='python')
x_temp2 = pd.read_csv('E:\\极客大学\\机器学习训练营\\作业\\毕业设计\\ML_chapter7_dataset\\final\\test_final.csv', engine='python')

In [6]:
print('xapply_temp1', x_temp1.head(5)) 
print('xapply_temp2', x_temp2.head(5))
# x_temp1为训练集、x_temp2为测试集

xapply_temp1    continuous_annual_inc  continuous_annual_inc_joint  continuous_delinq_2yrs  \
0                55000.0                          NaN                     0.0   
1                65000.0                          NaN                     1.0   
2                63000.0                      71000.0                     0.0   
3               104433.0                          NaN                     1.0   
4                34000.0                          NaN                     0.0   

   continuous_dti  continuous_dti_joint  continuous_fico_range_high  \
0            5.91                   NaN                       679.0   
1           16.06                   NaN                       719.0   
2           10.78                 13.85                       699.0   
3           25.37                   NaN                       699.0   
4           10.20                   NaN                       694.0   

   continuous_fico_range_low  continuous_funded_amnt  \
0                

In [7]:
print(type(x_temp1))

<class 'pandas.core.frame.DataFrame'>


In [9]:
train_new_item = x_temp1.copy()
test_new_item = x_temp2.copy()

In [10]:
train_new_item['discrete_2+4'] = (train_new_item['discrete_grade_2_one_hot'] + train_new_item['discrete_grade_4_one_hot'])
train_new_item['discrete_5+6'] = (train_new_item['discrete_grade_5_one_hot'] + train_new_item['discrete_grade_6_one_hot'])

test_new_item['discrete_2+4'] = (test_new_item['discrete_grade_2_one_hot'] + test_new_item['discrete_grade_4_one_hot'])
test_new_item['discrete_5+6'] = (test_new_item['discrete_grade_5_one_hot'] + test_new_item['discrete_grade_6_one_hot'])

In [11]:
train_new_item.head(5)

,continuous_annual_inc,continuous_annual_inc_joint,continuous_delinq_2yrs,continuous_dti,continuous_dti_joint,continuous_fico_range_high,continuous_fico_range_low,continuous_funded_amnt,continuous_funded_amnt_inv,continuous_inq_last_6mths,...,discrete_sub_grade_30_one_hot,discrete_sub_grade_31_one_hot,discrete_sub_grade_32_one_hot,discrete_sub_grade_33_one_hot,discrete_sub_grade_34_one_hot,discrete_sub_grade_35_one_hot,discrete_term_1_one_hot,discrete_term_2_one_hot,discrete_2+4,discrete_5+6
0,55000.0,NaN,0.0,5.91,NaN,679.0,675.0,3600.0,3600.0,1.0,...,0,0,0,0,0,0,1,0,0,0
1,65000.0,NaN,1.0,16.06,NaN,719.0,715.0,24700.0,24700.0,4.0,...,0,0,0,0,0,0,1,0,0,0
2,63000.0,71000.0,0.0,10.78,13.85,699.0,695.0,20000.0,20000.0,0.0,...,0,0,0,0,0,0,0,1,1,0
3,104433.0,NaN,1.0,25.37,NaN,699.0,695.0,10400.0,10400.0,3.0,...,0,0,0,0,0,0,0,1,0,0
4,34000.0,NaN,0.0,10.20,NaN,694.0,690.0,11950.0,11950.0,0.0,...,0,0,0,0,0,0,1,0,0,0


In [13]:
X_train_new_item = train_new_item.drop(columns=['loan_status']).values
Y_train_new_item = train_new_item['loan_status'].values.astype(int)
X_test_new_item = test_new_item.drop(columns=['loan_status']).values
Y_test_new_item = test_new_item['loan_status'].values.astype(int)

In [14]:
print(X_train_new_item)

[[5.50e+04      nan 0.00e+00 ... 0.00e+00 0.00e+00 0.00e+00]
 [6.50e+04      nan 1.00e+00 ... 0.00e+00 0.00e+00 0.00e+00]
 [6.30e+04 7.10e+04 0.00e+00 ... 1.00e+00 1.00e+00 0.00e+00]
 ...
 [5.10e+04      nan 0.00e+00 ... 0.00e+00 0.00e+00 0.00e+00]
 [4.55e+05      nan 0.00e+00 ... 0.00e+00 1.00e+00 0.00e+00]
 [5.80e+04      nan 0.00e+00 ... 0.00e+00 1.00e+00 0.00e+00]]


In [36]:
seed = 42 # for the same data division

kf = KFold(n_splits = 5, random_state=seed,shuffle=True)

In [37]:
five_fold_data = []

for train_index, eval_index in kf.split(X_train_new_item):
    x_train, x_eval = X_train_new_item[train_index], X_train_new_item[eval_index]
    y_train, y_eval = Y_train_new_item[train_index], Y_train_new_item[eval_index]
    
    five_fold_data.append([(x_train, y_train), (x_eval, y_eval)])

In [28]:
print(five_fold_data)

[[(array([[5.50e+04,      nan, 0.00e+00, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [6.50e+04,      nan, 1.00e+00, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [6.30e+04, 7.10e+04, 0.00e+00, ..., 1.00e+00, 1.00e+00, 0.00e+00],
       ...,
       [5.10e+04,      nan, 0.00e+00, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [4.55e+05,      nan, 0.00e+00, ..., 0.00e+00, 1.00e+00, 0.00e+00],
       [5.80e+04,      nan, 0.00e+00, ..., 0.00e+00, 1.00e+00, 0.00e+00]]), array([1, 1, 1, ..., 1, 1, 1])), (array([[3.40e+04,      nan, 0.00e+00, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [4.20e+04,      nan, 0.00e+00, ..., 1.00e+00, 0.00e+00, 0.00e+00],
       [6.00e+04,      nan, 0.00e+00, ..., 1.00e+00, 0.00e+00, 1.00e+00],
       ...,
       [3.64e+04,      nan, 1.00e+00, ..., 0.00e+00, 0.00e+00, 1.00e+00],
       [5.00e+04,      nan, 0.00e+00, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [1.00e+05,      nan, 0.00e+00, ..., 0.00e+00, 1.00e+00, 0.00e+00]]), array([1, 0, 0, ..., 0, 1, 1]))], [(array([[5.5

In [38]:
def get_model(param):
    model_list = []
    for idx, [(x_train, y_train), (x_eval, y_eval)] in enumerate(five_fold_data):
        print('{}-th model is training:'.format(idx))
        train_data = lgb.Dataset(x_train, label=y_train)
        validation_data = lgb.Dataset(x_eval, label=y_eval)
        bst = lgb.train(param, train_data, valid_sets=[validation_data])
        model_list.append(bst)
    return model_list

In [39]:
param_base = {'num_leaves': 31, 'objective': 'binary', 'metric': 'binary', 'num_round':1000}

param_fine_tuning = {'num_thread': 8,'num_leaves': 128, 'metric': 'binary', 'objective': 'binary', 'num_round': 1000, 
                     'learning_rate': 3e-3, 'feature_fraction': 0.6, 'bagging_fraction': 0.8}

In [40]:
# base param train
param_base_model = get_model(param_base)

# param fine tuning
param_fine_tuning_model = get_model(param_fine_tuning)

0-th model is training:


C:\Users\Administrator\Anaconda3\envs\py37\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `num_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Info] Number of positive: 31851, number of negative: 8149
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007298 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2587
[LightGBM] [Info] Number of data points in the train set: 40000, number of used features: 143
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.796275 -> initscore=1.363174
[LightGBM] [Info] Start training from score 1.363174
[1]	valid_0's binary_logloss: 0.448173
[2]	valid_0's binary_logloss: 0.405734
[3]	valid_0's binary_logloss: 0.372961
[4]	valid_0's binary_logloss: 0.34683
[5]	valid_0's binary_logloss: 0.325461
[6]	valid_0's binary_logloss: 0.307594
[7]	valid_0's binary_logloss: 0.292607
[8]	valid_0's binary_logloss: 0.279725
[9]	valid_0's binary_logloss: 0.268803
[10]	valid_0's binary_logloss: 0.259461
[11]	valid_0's binary_logloss: 0.251464
[12]	valid_

[193]	valid_0's binary_logloss: 0.201546
[194]	valid_0's binary_logloss: 0.201565
[195]	valid_0's binary_logloss: 0.20153
[196]	valid_0's binary_logloss: 0.201599
[197]	valid_0's binary_logloss: 0.201623
[198]	valid_0's binary_logloss: 0.201642
[199]	valid_0's binary_logloss: 0.201659
[200]	valid_0's binary_logloss: 0.201651
[201]	valid_0's binary_logloss: 0.201679
[202]	valid_0's binary_logloss: 0.201698
[203]	valid_0's binary_logloss: 0.201826
[204]	valid_0's binary_logloss: 0.201814
[205]	valid_0's binary_logloss: 0.201863
[206]	valid_0's binary_logloss: 0.201882
[207]	valid_0's binary_logloss: 0.201983
[208]	valid_0's binary_logloss: 0.201977
[209]	valid_0's binary_logloss: 0.202029
[210]	valid_0's binary_logloss: 0.20204
[211]	valid_0's binary_logloss: 0.202025
[212]	valid_0's binary_logloss: 0.201937
[213]	valid_0's binary_logloss: 0.20197
[214]	valid_0's binary_logloss: 0.201989
[215]	valid_0's binary_logloss: 0.202033
[216]	valid_0's binary_logloss: 0.202045
[217]	valid_0's bin

[407]	valid_0's binary_logloss: 0.206954
[408]	valid_0's binary_logloss: 0.207021
[409]	valid_0's binary_logloss: 0.20701
[410]	valid_0's binary_logloss: 0.20716
[411]	valid_0's binary_logloss: 0.207286
[412]	valid_0's binary_logloss: 0.207291
[413]	valid_0's binary_logloss: 0.207297
[414]	valid_0's binary_logloss: 0.207358
[415]	valid_0's binary_logloss: 0.20743
[416]	valid_0's binary_logloss: 0.207436
[417]	valid_0's binary_logloss: 0.207495
[418]	valid_0's binary_logloss: 0.207559
[419]	valid_0's binary_logloss: 0.207527
[420]	valid_0's binary_logloss: 0.207526
[421]	valid_0's binary_logloss: 0.207576
[422]	valid_0's binary_logloss: 0.207617
[423]	valid_0's binary_logloss: 0.20765
[424]	valid_0's binary_logloss: 0.207726
[425]	valid_0's binary_logloss: 0.207773
[426]	valid_0's binary_logloss: 0.207791
[427]	valid_0's binary_logloss: 0.207834
[428]	valid_0's binary_logloss: 0.207819
[429]	valid_0's binary_logloss: 0.207793
[430]	valid_0's binary_logloss: 0.20783
[431]	valid_0's binar

...

[151]	valid_0's binary_logloss: 0.346108
[152]	valid_0's binary_logloss: 0.345418
[153]	valid_0's binary_logloss: 0.344731
[154]	valid_0's binary_logloss: 0.34405
[155]	valid_0's binary_logloss: 0.343369
[156]	valid_0's binary_logloss: 0.342687
[157]	valid_0's binary_logloss: 0.342011
[158]	valid_0's binary_logloss: 0.341348
[159]	valid_0's binary_logloss: 0.34068
[160]	valid_0's binary_logloss: 0.340019
[161]	valid_0's binary_logloss: 0.339365
[162]	valid_0's binary_logloss: 0.338711
[163]	valid_0's binary_logloss: 0.338065
[164]	valid_0's binary_logloss: 0.337416
[165]	valid_0's binary_logloss: 0.336776
[166]	valid_0's binary_logloss: 0.336139
[167]	valid_0's binary_logloss: 0.336004
[168]	valid_0's binary_logloss: 0.335368
[169]	valid_0's binary_logloss: 0.334741
[170]	valid_0's binary_logloss: 0.334116
[171]	valid_0's binary_logloss: 0.333497
[172]	valid_0's binary_logloss: 0.33287
[173]	valid_0's binary_logloss: 0.332248
[174]	valid_0's binary_logloss: 0.331633
[175]	valid_0's bin

[356]	valid_0's binary_logloss: 0.26457
[357]	valid_0's binary_logloss: 0.264277
[358]	valid_0's binary_logloss: 0.263995
[359]	valid_0's binary_logloss: 0.263705
[360]	valid_0's binary_logloss: 0.26342
[361]	valid_0's binary_logloss: 0.26314
[362]	valid_0's binary_logloss: 0.262855
[363]	valid_0's binary_logloss: 0.262571
[364]	valid_0's binary_logloss: 0.262292
[365]	valid_0's binary_logloss: 0.262017
[366]	valid_0's binary_logloss: 0.261742
[367]	valid_0's binary_logloss: 0.261465
[368]	valid_0's binary_logloss: 0.261191
[369]	valid_0's binary_logloss: 0.26092
[370]	valid_0's binary_logloss: 0.260655
[371]	valid_0's binary_logloss: 0.260379
[372]	valid_0's binary_logloss: 0.26011
[373]	valid_0's binary_logloss: 0.259845
[374]	valid_0's binary_logloss: 0.259577
[375]	valid_0's binary_logloss: 0.25931
[376]	valid_0's binary_logloss: 0.259035
[377]	valid_0's binary_logloss: 0.258773
[378]	valid_0's binary_logloss: 0.258512
[379]	valid_0's binary_logloss: 0.25825
[380]	valid_0's binary_

[556]	valid_0's binary_logloss: 0.228151
[557]	valid_0's binary_logloss: 0.228025
[558]	valid_0's binary_logloss: 0.227892
[559]	valid_0's binary_logloss: 0.227863
[560]	valid_0's binary_logloss: 0.227731
[561]	valid_0's binary_logloss: 0.227602
[562]	valid_0's binary_logloss: 0.22747
[563]	valid_0's binary_logloss: 0.227341
[564]	valid_0's binary_logloss: 0.227215
[565]	valid_0's binary_logloss: 0.227087
[566]	valid_0's binary_logloss: 0.22706
[567]	valid_0's binary_logloss: 0.226932
[568]	valid_0's binary_logloss: 0.226805
[569]	valid_0's binary_logloss: 0.226686
[570]	valid_0's binary_logloss: 0.22656
[571]	valid_0's binary_logloss: 0.226438
[572]	valid_0's binary_logloss: 0.226312
[573]	valid_0's binary_logloss: 0.226187
[574]	valid_0's binary_logloss: 0.226058
[575]	valid_0's binary_logloss: 0.225938
[576]	valid_0's binary_logloss: 0.225816
[577]	valid_0's binary_logloss: 0.225695
[578]	valid_0's binary_logloss: 0.225567
[579]	valid_0's binary_logloss: 0.225544
[580]	valid_0's bin

[762]	valid_0's binary_logloss: 0.211451
[763]	valid_0's binary_logloss: 0.21144
[764]	valid_0's binary_logloss: 0.211376
[765]	valid_0's binary_logloss: 0.211318
[766]	valid_0's binary_logloss: 0.211259
[767]	valid_0's binary_logloss: 0.211199
[768]	valid_0's binary_logloss: 0.211132
[769]	valid_0's binary_logloss: 0.211075
[770]	valid_0's binary_logloss: 0.211059
[771]	valid_0's binary_logloss: 0.211001
[772]	valid_0's binary_logloss: 0.210947
[773]	valid_0's binary_logloss: 0.210882
[774]	valid_0's binary_logloss: 0.210824
[775]	valid_0's binary_logloss: 0.210764
[776]	valid_0's binary_logloss: 0.210706
[777]	valid_0's binary_logloss: 0.210644
[778]	valid_0's binary_logloss: 0.210584
[779]	valid_0's binary_logloss: 0.210524
[780]	valid_0's binary_logloss: 0.210469
[781]	valid_0's binary_logloss: 0.210414
[782]	valid_0's binary_logloss: 0.210357
[783]	valid_0's binary_logloss: 0.210299
[784]	valid_0's binary_logloss: 0.210243
[785]	valid_0's binary_logloss: 0.210187
[786]	valid_0's b

[967]	valid_0's binary_logloss: 0.203563
[968]	valid_0's binary_logloss: 0.203529
[969]	valid_0's binary_logloss: 0.203499
[970]	valid_0's binary_logloss: 0.203471
[971]	valid_0's binary_logloss: 0.203469
[972]	valid_0's binary_logloss: 0.203444
[973]	valid_0's binary_logloss: 0.203439
[974]	valid_0's binary_logloss: 0.20341
[975]	valid_0's binary_logloss: 0.20338
[976]	valid_0's binary_logloss: 0.20335
[977]	valid_0's binary_logloss: 0.203327
[978]	valid_0's binary_logloss: 0.2033
[979]	valid_0's binary_logloss: 0.203271
[980]	valid_0's binary_logloss: 0.203245
[981]	valid_0's binary_logloss: 0.203215
[982]	valid_0's binary_logloss: 0.203188
[983]	valid_0's binary_logloss: 0.203155
[984]	valid_0's binary_logloss: 0.203132
[985]	valid_0's binary_logloss: 0.203112
[986]	valid_0's binary_logloss: 0.20309
[987]	valid_0's binary_logloss: 0.203063
[988]	valid_0's binary_logloss: 0.203034
[989]	valid_0's binary_logloss: 0.203035
[990]	valid_0's binary_logloss: 0.203009
[991]	valid_0's binary

In [41]:
def test_model(model_list):
    data = X_test_new_item
    five_fold_pred = np.zeros((5, len(X_test_new_item))) 
    for i, bst in enumerate(model_list):
        ypred = bst.predict(data, num_iteration=bst.best_iteration)
        five_fold_pred[i] = ypred
    ypred_mean = (five_fold_pred.mean(axis=-2)>0.5).astype(int)
    return accuracy_score(ypred_mean, Y_test_new_item)

In [42]:
base_score = test_model(param_base_model)
fine_tuning_score = test_model(param_fine_tuning_model)

print('base: {}, fine tuning: {}'.format(base_score, fine_tuning_score))

base: 0.91568, fine tuning: 0.91758


### 原baseline的数据是：base :0.91552, fine tuning: 0.91756

In [51]:
print('base提升：%.6f%%' %((0.91568-0.91552)/0.91522))

base提升：0.000175%


In [52]:
print('fine tuning提升：%.6f%%' %((0.91758-0.91756)/0.91756))

fine tuning提升：0.000022%


### 故采用衍生变量比原模型：base提升：0.000175%, fine tuning提升：0.000022%

## 第二稿准备采用tabnet方式进一步提升